# Core Component: Simulation Harness

The `SimulationHarness` is a powerful tool for setting up, running, and testing simulations. It manages the components, their connections, and the main simulation loop. It essentially allows you to build a water system from its constituent parts.

### Key Responsibilities:
1. **Component Management**: Holds all the physical components (`Reservoir`, `Pipe`, etc.) and agents in the simulation.
2. **Topology Definition**: Manages the connections between physical components (e.g., `reservoir_1 -> pipe_1 -> gate_1`).
3. **Topological Sort**: Before running, it sorts the components into the correct execution order to ensure that water flows logically from upstream to downstream.
4. **Simulation Loop**: Provides methods (`run_simulation`, `run_mas_simulation`) that execute the main simulation loop, stepping through time and calling the components' `step` methods in the correct order.
5. **History Tracking**: Automatically records the state of all components at every time step in the `harness.history` attribute.

## Tutorial: Building a Simple Simulation

This notebook provides a step-by-step tutorial on how to use the `SimulationHarness` to build and run a simple simulation of a reservoir emptying through a pipe and a gate.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.pipe import Pipe
from swp.simulation_identification.physical_objects.gate import Gate
from swp.core.interfaces import State, Parameters

print("--- Step 1: Create a SimulationHarness ---")
simulation_config = {'duration': 600, 'dt': 1.0}
harness = SimulationHarness(config=simulation_config)

print("\n--- Step 2: Define the Physical Components ---")
reservoir = Reservoir(name="res1", initial_state=State({'volume': 25e6, 'water_level': 15.0}), parameters=Parameters({'surface_area': 1.5e6}))
pipe = Pipe(name="pipe1", initial_state=State({'outflow': 0}), parameters=Parameters({'length': 1000, 'diameter': 1.5, 'friction_factor': 0.02}))
gate = Gate(name="g1", initial_state=State({'opening': 0.3}), parameters=Parameters({'width': 10, 'max_opening': 1.0, 'discharge_coefficient': 0.6}))

print("\n--- Step 3: Add Components to the Harness ---")
harness.add_component(reservoir)
harness.add_component(pipe)
harness.add_component(gate)

print("\n--- Step 4: Define the Topology (Connections) ---")
harness.add_connection("res1", "pipe1")
harness.add_connection("pipe1", "g1")

print("\n--- Step 5: Build the Simulation ---")
harness.build()

print("\n--- Step 6: Run the Simulation ---")
# Redirecting verbose print output to a log file for cleaner notebook output
import sys
original_stdout = sys.stdout
with open('harness_tutorial_log.txt', 'w') as f:
    sys.stdout = f
    harness.run_simulation()
sys.stdout = original_stdout
print("Simulation finished. Results are in harness.history.")

## Step 7: Analyze the Results

The `harness.history` attribute now contains a list of dictionaries, where each dictionary is a snapshot of the entire system state at a single point in time. We can easily process this with Pandas and plot the results.

In [ ]:
# Extract data from history
time = [h['time'] for h in harness.history]
res_level = [h['res1']['water_level'] for h in harness.history]
pipe_outflow = [h['pipe1']['outflow'] for h in harness.history]
gate_outflow = [h['g1']['outflow'] for h in harness.history]

# Create a DataFrame
df = pd.DataFrame({
    'Time': time,
    'Reservoir Level': res_level,
    'Pipe Outflow': pipe_outflow,
    'Gate Outflow': gate_outflow
})

print("First 5 steps of simulation history:")
print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

ax1.plot(df['Time'], df['Reservoir Level'], label='Reservoir Level')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('Simulation Harness Tutorial Results')
ax1.grid(True)
ax1.legend()

ax2.plot(df['Time'], df['Pipe Outflow'], label='Pipe Outflow')
ax2.plot(df['Time'], df['Gate Outflow'], label='Gate Outflow', linestyle='--')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Outflow (m^3/s)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()